In [12]:
import numpy as np
import json
from benlp.llms import Chat

In [35]:
def initialize_simplex_tableau(c, A, b):
    num_constraints = len(A)
    num_variables = len(c)

    # Create the initial tableau
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))

    # Set the coefficients of the objective function
    tableau[0, :num_variables] = -c

    # Set the coefficients of the constraints
    tableau[1:, :num_variables] = A

    # Set the coefficients of the slack variables
    for i in range(num_constraints):
        tableau[i + 1, num_variables + i] = 1

    # Set the right-hand side values
    tableau[1:, -1] = b

    return tableau

def pivot_tableau(tableau, pivot_row, pivot_col):
    # Normalize the pivot row
    tableau[pivot_row] /= tableau[pivot_row, pivot_col]

    # Update the other rows
    for i in range(tableau.shape[0]):
        if i != pivot_row:
            tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

def simplex_algorithm(c, A, b, minimize=True):
    if not minimize:
        c = -c

    tableau = initialize_simplex_tableau(c, A, b)

    while True:
        # Find the pivot column
        pivot_col = np.argmin(tableau[0, :-1])

        # If all coefficients are non-negative, we found the optimal solution
        if tableau[0, pivot_col] >= 0:
            break

        # Find the pivot row
        positive_rows = tableau[1:, pivot_col] > 0
        pivot_row = np.argmin(tableau[1:, -1][positive_rows] / tableau[1:, pivot_col][positive_rows]) + 1

        # If there is no pivot row, the problem is unbounded
        if not np.any(positive_rows):
            raise ValueError("The problem is unbounded.")

        # Pivot the tableau
        pivot_tableau(tableau, pivot_row, pivot_col)

    # Extract the optimal solution
    solution = np.zeros(c.shape)
    for i in range(len(c)):
        row = np.where(tableau[:, i] == 1)[0]
        if row.size > 0 and tableau[row, -1].size > 0:
            solution[i] = tableau[row, -1][0]

    return solution, -tableau[0, -1]

In [30]:
def args_parser(json_string):
    data = json.loads(json_string)
    c = np.array(data["c"])
    A = np.array(data["A"])
    b = np.array(data["b"])
    minimize = data["minimize"]
    args = c, A, b
    kwargs = {"minimize": minimize}
    return args, kwargs

In [24]:
code_string = """

def initialize_simplex_tableau(c, A, b, minimize=True):
    if not minimize:
        c = -c

    num_constraints = len(A)
    num_variables = len(c)

    # Create the initial tableau
    tableau = np.zeros((num_constraints + 1, num_variables + num_constraints + 1))

    # Set the coefficients of the objective function
    tableau[0, :num_variables] = -c

    # Set the coefficients of the constraints
    tableau[1:, :num_variables] = A

    # Set the coefficients of the slack variables
    for i in range(num_constraints):
        tableau[i + 1, num_variables + i] = 1

    # Set the right-hand side values
    tableau[1:, -1] = b

    return tableau

def pivot_tableau(tableau, pivot_row, pivot_col):
    # Normalize the pivot row
    tableau[pivot_row] /= tableau[pivot_row, pivot_col]

    # Update the other rows
    for i in range(tableau.shape[0]):
        if i != pivot_row:
            tableau[i] -= tableau[i, pivot_col] * tableau[pivot_row]

def simplex_algorithm(c, A, b):
    tableau = initialize_simplex_tableau(c, A, b)

    while True:
        # Find the pivot column
        pivot_col = np.argmin(tableau[0, :-1])

        # If all coefficients are non-negative, we found the optimal solution
        if tableau[0, pivot_col] >= 0:
            break

        # Find the pivot row
        positive_rows = tableau[1:, pivot_col] > 0
        pivot_row = np.argmin(tableau[1:, -1][positive_rows] / tableau[1:, pivot_col][positive_rows]) + 1

        # If there is no pivot row, the problem is unbounded
        if not np.any(positive_rows):
            raise ValueError("The problem is unbounded.")

        # Pivot the tableau
        pivot_tableau(tableau, pivot_row, pivot_col)

    # Extract the optimal solution
    solution = np.zeros(c.shape)
    for i in range(len(c)):
        row = np.where(tableau[:, i] == 1)[0]
        if row.size > 0 and tableau[row, -1].size > 0:
            solution[i] = tableau[row, -1][0]

    return solution, -tableau[0, -1]
"""

In [38]:
word_problems = [
    """
    The minimum daily req for vitamin A and b are 100 and 200 respectively.
    pizza has 10 vitamin A and 30 vitamin B. root beer has 20 vitamin A and 10 vitamin b. pizza costs $10 and root beet costs $5.
    """,
    
    """A factory produces two types of products, A and B. Each product A requires 2 hours of labor and 3 hours of machine time, while each product B requires 4 hours of labor and 1 hour of machine time. The factory has a total of 40 hours of labor and 30 hours of machine time available per day. The profit for each product A is $5, and the profit for each product B is $4. How many units of each product should the factory produce to maximize its daily profit?
    """
]

prompt = f"""Given the word problem: {word_problems[0]}
and the code:
{code_string}""" + """
Output a json object with the required arguments to run the simplex algorithm. 
1. Decide if the problem is a maximization or minimization problem.
2. Decide if a non-negative constraint is needed.
3. Do everything else.

EXAMPLE OUTPUT:
{
    "c": [5, 4],
    "A": [[2, 4], [3, 1]],
    "b": [40, 30],
    "minimize": false
}
Output the json only. Make sure to include a non negative constraint if needed.
"""

In [39]:
chat = Chat()
args = chat(prompt)['response']
print("Unparsed args:", args)

parsed_args, parsed_kwargs = args_parser(args)
print("Parsed args:")
for arg in parsed_args:
    print(arg)
print("\n\n")

print("Parsed kwargs:")
for key, value in parsed_kwargs.items():
    print(key, value)
print("\n\n")

solution, optimal_value = simplex_algorithm(*parsed_args, **parsed_kwargs)
print("Solution:", solution)
print("Optimal value:", optimal_value)

Unparsed args: {
    "c": [10, 5],
    "A": [[10, 20], [30, 10]],
    "b": [100, 200],
    "minimize": true
}
Parsed args:
[10  5]
[[10 20]
 [30 10]]
[100 200]



Parsed kwargs:
minimize True



Solution: [6. 2.]
Optimal value: -70.0
